In [1]:
import pandas as pd
import heapq
import folium

In [2]:
in_path = 'in'
cpp_data = pd.read_csv(f"{in_path}/awards_2026.csv")
iata_icao_data = pd.read_csv(f"{in_path}/iata-icao.csv")
per_diem_data = pd.read_csv(f"{in_path}/FY2026_PerDiemMasterRatesFile.csv")

In [3]:
# Filter the DataFrame to only include routes where both origin and destination are in the United States
cpp_data = cpp_data[
    (cpp_data['ORIGIN_COUNTRY'] == 'UNITED STATES') &
    (cpp_data['DESTINATION_COUNTRY'] == 'UNITED STATES')
]
# Reset the index after filtering
cpp_data.reset_index(drop = True, inplace = True)

In [4]:
# cpp_data.nunique()
# cpp_data.isnull().sum()
# cpp_data.head()
# cpp_data.describe()
# cpp_data.info()
# cpp_data.columns
# cpp_data[cpp_data['ORIGIN_AIRPORT_ABBREV'] == 'SEA']
per_diem_data.head()

,ID,STATE,DESTINATION,COUNTY/LOCATION DEFINED,SEASON BEGIN,SEASON END,FY26 Lodging Rate,FY26 M&IE,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1,AL,Birmingham,Jefferson,NaN,NaN,$ 126,$ 80,NaN,NaN,
1,2,AL,Gulf Shores,Baldwin,October 1,February 28,$ 134,$ 74,NaN,NaN,NaN
2,2,AL,Gulf Shores,Baldwin,March 1,May 31,$ 163,$ 74,NaN,NaN,NaN
3,2,AL,Gulf Shores,Baldwin,June 1,July 31,$ 216,$ 74,NaN,NaN,NaN
4,2,AL,Gulf Shores,Baldwin,August 1,September 30,$ 134,$ 74,NaN,NaN,NaN


In [5]:
# Filter rows where SEA appears in ORIGIN_AIRPORT_ABBREV or DESTINATION_AIRPORT_ABBREV
node = 'MDW'
# Filter rows where SEA appears in ORIGIN_AIRPORT_ABBREV or DESTINATION_AIRPORT_ABBREV
node_df = cpp_data[(cpp_data['ORIGIN_AIRPORT_ABBREV'] == node) | (cpp_data['DESTINATION_AIRPORT_ABBREV'] == node)]
print(node_df)

      ITEM_NUM  AWARD_YEAR ORIGIN_AIRPORT_ABBREV DESTINATION_AIRPORT_ABBREV  \
24          27        2026                   ABQ                        MDW   
67          91        2026                   ALB                        MDW   
198        289        2026                   ATL                        MDW   
298        432        2026                   AUS                        MDW   
355        513        2026                   BDL                        MDW   
...        ...         ...                   ...                        ...   
9659     13970        2026                   MDW                        MGM   
9660     13971        2026                   MDW                        PBI   
9661     13972        2026                   MDW                        RNO   
9662     13973        2026                   MDW                        SNA   
9663     13974        2026                   MDW                        VPS   

     ORIGIN_CITY_NAME ORIGIN_STATE ORIGIN_COUNTRY D

In [6]:
def get_fares(start, goal):
    # Filter for matching pairs in either direction
    match = cpp_data[((cpp_data['ORIGIN_AIRPORT_ABBREV'] == start) & (cpp_data['DESTINATION_AIRPORT_ABBREV'] == goal)) |
                     ((cpp_data['ORIGIN_AIRPORT_ABBREV'] == goal) & (cpp_data['DESTINATION_AIRPORT_ABBREV'] == start))]
    
    if not match.empty:
        # Extract all columns ending with '_FARE'
        fare_columns = [col for col in match.columns if col.endswith('_FARE')]
        fares = {col: match[col].iloc[0] for col in fare_columns}
        
        return fares
    else:
        print(f"No direct fares found between {start} and {goal}.")
        return None

# Example usage of the fare lookup
start = 'FLL'
goal = 'RDU'
fares = get_fares(start, goal)
if fares is not None:
    print(f"Direct fares between {start} and {goal}: {fares}")


Direct fares between FLL and RDU: {'YCA_FARE': np.int64(86), '_CA_FARE': np.int64(64), 'BUSINESS_FARE': np.int64(0), '_CP_FARE': np.int64(0)}


In [7]:
# Heuristic function for A* (currently using path cost only)
def heuristic(current_cost, weight):
    return current_cost + weight

In [8]:
# Build the graph using ORIGIN and DESTINATION airport codes with YCA_FARE as edge weights
graph = {}
for _, row in cpp_data.iterrows():
    start = row['ORIGIN_AIRPORT_ABBREV']
    goal = row['DESTINATION_AIRPORT_ABBREV']
    weight = row['YCA_FARE']
    
    if start not in graph:
        graph[start] = []
    if goal not in graph:
        graph[goal] = []
    
    # Assuming bidirectional flights
    graph[start].append((goal, weight))
    graph[goal].append((start, weight))

# A* Algorithm Implementation
def a_star(graph, start, goal):
    queue = []
    heapq.heappush(queue, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}
    
    while queue:
        current_priority, current_node = heapq.heappop(queue)
        
        if current_node == goal:
            break
        
        for neighbor, weight in graph.get(current_node, []):
            new_cost = heuristic(cost_so_far[current_node], weight)
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost  # No heuristic used here
                heapq.heappush(queue, (priority, neighbor))
                came_from[neighbor] = current_node
    
    # Reconstruct path
    path = []
    node = goal
    while node is not None:
        path.append(node)
        node = came_from.get(node)
    path.reverse()
    
    return path, cost_so_far.get(goal, float('inf'))

# Example run: CDC to MID
start = 'FLL'
goal = 'RDU'
path, total_cost = a_star(graph, start, goal)
print(f"Path from {start} to {goal}: {path}")
print(f"Total YCA Fare: ${total_cost}")

Path from FLL to RDU: ['FLL', 'RDU']
Total YCA Fare: $86


In [10]:
# Map IATA codes to their coordinates
iata_coords = {row['iata']: (row['latitude'], row['longitude']) for _, row in iata_icao_data.iterrows()}

def plot_path_on_map(path):
    # Initialize the map centered on the first airport in the path
    if path[0] in iata_coords:
        start_coords = iata_coords[path[0]]
        route_map = folium.Map(location = start_coords, zoom_start = 5)
    else:
        route_map = folium.Map(zoom_start = 5)
    
    # Plot each airport in the path
    for i in range(len(path) - 1):
        origin = path[i]
        destination = path[i + 1]
        if origin in iata_coords and destination in iata_coords:
            folium.Marker(iata_coords[origin], popup = origin, icon = folium.Icon(color = 'green')).add_to(route_map)
            folium.Marker(iata_coords[destination], popup=destination, icon=folium.Icon(color = 'red')).add_to(route_map)
            folium.PolyLine(locations=[iata_coords[origin], iata_coords[destination]], color = 'blue').add_to(route_map)
    
    return route_map

# Plot the path on the map
route_map = plot_path_on_map(path)
out_path = "out"
route_map.save(f"{out_path}/flight_route_map.html")
print("Map has been saved as 'flight_route_map.html'.")

Map has been saved as 'flight_route_map.html'.


In [11]:
import heapq

# Build the graph using ORIGIN and DESTINATION airport codes with YCA_FARE as edge weights
graph = {}
for _, row in cpp_data.iterrows():
    start = row['ORIGIN_AIRPORT_ABBREV']
    goal = row['DESTINATION_AIRPORT_ABBREV']
    weight = row['YCA_FARE']
    
    if start not in graph:
        graph[start] = []
    if goal not in graph:
        graph[goal] = []
    
    # Assuming bidirectional flights
    graph[start].append((goal, weight))
    graph[goal].append((start, weight))

# A* Algorithm Implementation
def a_star(graph, start, goal):
    queue = []
    heapq.heappush(queue, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}
    
    while queue:
        current_priority, current_node = heapq.heappop(queue)
        
        if current_node == goal:
            break
        
        for neighbor, weight in graph.get(current_node, []):
            new_cost = heuristic(cost_so_far[current_node], weight)
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost
                heapq.heappush(queue, (priority, neighbor))
                came_from[neighbor] = current_node
    
    # Reconstruct path
    path = []
    node = goal
    while node is not None:
        path.append(node)
        node = came_from.get(node)
    path.reverse()
    
    return path, cost_so_far.get(goal, float('inf'))

# Function to find the optimal meeting point for multiple travelers
def optimal_meeting_point(graph, start_points):
    min_total_cost = float('inf')
    best_meeting_point = None
    
    for airport in graph.keys():
        total_cost = 0
        for start in start_points:
            _, cost = a_star(graph, start, airport)
            total_cost += cost
        
        if total_cost < min_total_cost:
            min_total_cost = total_cost
            best_meeting_point = airport
    
    return best_meeting_point, min_total_cost

# Function to print the paths from multiple starting points to a target airport
def print_paths_to_destination(graph, start_points, destination):
    for start in start_points:
        path, cost = a_star(graph, start, destination)
        print(f"Path from {start} to {destination}: {path}")
        print(f"Total YCA Fare from {start} to {destination}: ${cost}")

# Example run with a list of starting points
start_points = ['DCA', 'FLL', 'ORD', 'RDU']
# start_points = ['DCA', 'MIA', 'MDW', 'RDU', 'DEN']
# start_points = ['ATL', 'DFW', 'DEN', 'LIT', 'PIT', 'MIA', 'BUF', 'DCA']
# start_points = ['ATL', 'DFW', 'DEN', 'LIT', 'PIT', 'MIA', 'BUF']
# start_points = ['BDL', 'MSP', 'BWI', 'RDU', 'ORD', 'DCA', 'IAD', 'MIA', 'FLL'] # DS
# start_points = ['ABE', 'OKC', 'SEA', 'BWI']
# start_points = ['ABE', 'OKC']
meeting_point, total_cost = optimal_meeting_point(graph, start_points)
print(f"Optimal meeting point for {start_points}: {meeting_point}")
print(f"Total YCA Fare: ${total_cost}")

print_paths_to_destination(graph, start_points, meeting_point)

Optimal meeting point for ['DCA', 'FLL', 'ORD', 'RDU']: RDU
Total YCA Fare: $432
Path from DCA to RDU: ['DCA', 'BOS', 'RDU']
Total YCA Fare from DCA to RDU: $175
Path from FLL to RDU: ['FLL', 'RDU']
Total YCA Fare from FLL to RDU: $86
Path from ORD to RDU: ['ORD', 'RDU']
Total YCA Fare from ORD to RDU: $171
Path from RDU to RDU: ['RDU']
Total YCA Fare from RDU to RDU: $0


In [12]:
# Function to compute paths, show both A* and direct paths, and print both YCA fares
def print_paths_to_destination_with_direct_fares(graph, start_points, destination):
    results = []
    for start in start_points:
        path, computed_cost = a_star(graph, start, destination)

        # Check if a direct fare exists between start and destination
        direct_fares = get_fares(start, destination)
        if direct_fares and 'YCA_FARE' in direct_fares:
            direct_cost = direct_fares['YCA_FARE']
            direct_path = [start, destination]  # Direct path
        else:
            direct_cost = None  # No direct fare available
            direct_path = None

        results.append((start, path, computed_cost, direct_path, direct_cost))
    
    return results

# Example run with start points
start_points = ['IAD', 'MIA', 'MDW', 'RDU'] # 629
# start_points = ['IAD', 'MIA', 'ORD', 'RDU'] # 559
# start_points = ['DCA', 'MIA', 'ORD', 'RDU'] # 464
# start_points = ['IAD', 'FLL', 'ORD', 'RDU'] # 543
# start_points = ['DCA', 'FLL', 'ORD', 'RDU'] # $1 difference! 434

# Compute A* optimal meeting point
optimal_meeting_point_result, optimal_total_cost = optimal_meeting_point(graph, start_points)

# Compute meeting point and total cost using preferred direct fares
paths_results_with_direct_fares = print_paths_to_destination_with_direct_fares(graph, start_points, optimal_meeting_point_result)

direct_fare_total_cost = sum(cost for _, _, cost, _, cost in paths_results_with_direct_fares if cost is not None)

# Print both solutions
optimal_meeting_point_result, optimal_total_cost, (optimal_meeting_point_result, direct_fare_total_cost)

# Display both paths and fares
for start, path, computed_cost, direct_path, direct_cost in paths_results_with_direct_fares:
    print(f"From {start} to {optimal_meeting_point_result}:")
    print(f"  A* Path: {path} | A* Fare: ${computed_cost}")
    if direct_path:
        print(f"  Direct Path: {direct_path} | Direct Fare: ${direct_cost}")
    print()

# Print final total costs for both approaches
print(f"Total A* Optimal Fare: ${optimal_total_cost}")
print(f"Total YCA Direct Fare (if preferred): ${direct_fare_total_cost}")

No direct fares found between MIA and MIA.
From IAD to MIA:
  A* Path: ['IAD', 'MIA'] | A* Fare: $151
  Direct Path: ['IAD', 'MIA'] | Direct Fare: $151

From MIA to MIA:
  A* Path: ['MIA'] | A* Fare: $0

From MDW to MIA:
  A* Path: ['MDW', 'ATL', 'MIA'] | A* Fare: $287
  Direct Path: ['MDW', 'MIA'] | Direct Fare: $473

From RDU to MIA:
  A* Path: ['RDU', 'MIA'] | A* Fare: $214
  Direct Path: ['RDU', 'MIA'] | Direct Fare: $214

Total A* Optimal Fare: $652
Total YCA Direct Fare (if preferred): $838
